In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = os.path.join(os.getcwd(), '.kaggle')
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!kaggle datasets download -d xmaulana/psychikadataset-7b

In [ ]:
import zipfile
with zipfile.ZipFile('psychikadataset-7b.zip', 'r') as zip_ref:
    zip_ref.extractall('psychikadataset-7b')

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd
import torch

In [ ]:
# Cek device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Load model dan tokenizer
model_name = "Qwen/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# Konfigurasi Scaled Dot-Product Attention jika diperlukan (optional)
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_math_sdp(True)
torch.backends.cuda.enable_mem_efficient_sdp(False)

# Aktifkan gradient checkpointing
model.gradient_checkpointing_enable()
model.config.use_cache = False

# LoRA config
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["c_attn", "q_proj", "v_proj"],  # sesuaikan jika error
)

# Terapkan LoRA ke model
model = get_peft_model(model, peft_config)
model.to(device)

# Load dataset
df = pd.read_csv("psychikadataset-7b/data.csv")

# Format sesuai template chat Qwen
def format_chat_template(row):
    chat = [
        {"role": "user", "content": row["Context"]},
        {"role": "assistant", "content": row["Response"]}
    ]
    return tokenizer.apply_chat_template(chat, tokenize=False)

df["text"] = df.apply(format_chat_template, axis=1)

# Convert ke HuggingFace Dataset
dataset = Dataset.from_pandas(df[["text"]])

# Tokenisasi
def tokenize(sample):
    return tokenizer(sample["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# TrainingArguments
training_args = TrainingArguments(
    output_dir="./qwen-psychika-lora",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_steps=99999999,
    save_steps=100,
    save_total_limit=1,
    fp16=True,  # GUNAKAN fp16 untuk T4
    report_to=[],
    logging_strategy="no"
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator,
)

# Mulai training
trainer.train()

# Simpan model dan tokenizer
model.save_pretrained("qwen-psychika-lora")
tokenizer.save_pretrained("qwen-psychika-lora")

In [ ]:
!zip -r qwen-psychika-lora.zip qwen-psychika-lora

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

device = "cpu"
base_model_path = "./qwen-1.5-1.8B-local"  # folder lokal
adapter_path = "./qwen-psychika-lora"  # LoRA folder lokal

tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(base_model_path, trust_remote_code=True)

model = PeftModel.from_pretrained(base_model, adapter_path).to(device)
model.eval()

# Inisialisasi chat history
chat_history = [
    {"role": "system", "content": "Kamu adalah seorang profesional psikiater yang sedang memeriksa pasien yang sedang mengalami tekanan mental."}
]

print("Mulai percakapan dengan Qwen Psychika Chat (ketik 'exit' untuk keluar):")

# Loop percakapan
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    chat_history.append({"role": "user", "content": user_input})

    # Format prompt
    chat_prompt = tokenizer.apply_chat_template(chat_history, tokenize=False)

    # Tokenisasi dan pindahkan ke device
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Ambil token output baru saja (tanpa prompt)
    prompt_len = inputs["input_ids"].shape[1]
    generated_tokens = outputs[0][prompt_len:]
    reply = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    print("AI:", reply)

    chat_history.append({"role": "assistant", "content": reply})